In [1]:
# -*- coding: utf-8 -*
import os
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.backend import tensorflow_backend
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import scipy.linalg as LA
import cv2
import tensorflow as tf

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [4]:
X_eva = []
Y_eva = []

#テストデータの所得
print("人工物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/jinkou/'):
    warnings.filterwarnings('ignore')
    img_eva = img_to_array(load_img(filepath, grayscale=True, target_size=(128, 128)))        
    X_eva.append(img_eva)
    Y_eva.append(0) # 教師データ（正解）

print("人工物画像のテストデータを取得しました")

    
# 学習データの取得（非正解画像）
print("自然物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/sizen/'):
    img_eva = img_to_array(load_img(filepath, grayscale=True, target_size=(128, 128)))
    X_eva.append(img_eva)
    Y_eva.append(1) # 教師データ（正解）

print("自然物画像のテストデータを取得しました")

# NumPy配列に変換
X_eva = np.asarray(X_eva)
Y_eva = np.asarray(Y_eva)


#X = X.reshape(X.shape[0], 128, 128)   
#X_eva = X_eva.reshape(X_eva.shape[0], 128, 128)
    

# float32型に変換
X_eva = X_eva.astype('float32')


# 正規化(0～1)
X_eva = X_eva / 255.0


人工物画像のテストデータを取得してます...
人工物画像のテストデータを取得しました
自然物画像のテストデータを取得してます...
自然物画像のテストデータを取得しました


In [5]:
print("テストデータをzca白色化しています...")
#テストデータにzca白色化
datagen = ImageDataGenerator(zca_whitening=True)
g = datagen.flow(X_eva, Y_eva, X_eva.shape[0], shuffle=False)
x_test, y_test = g.next()
print(x_test.shape)
print(Y_eva.shape)
print("テストデータをzca白色化しました")
    
# 学習用データとテストデータに分割
#x_test = x_zcaw_eva
#y_test = Y_eva

テストデータをzca白色化しています...
(2400, 128, 128, 1)
(2400,)
テストデータをzca白色化しました


In [6]:
batch_size = 32 # バッチサイズ(データサイズ)
num_classes = 2 # 分類クラス数(今回は0～9の手書き文字なので10)
epochs = 100 # エポック数(学習の繰り返し回数)
ave = 0
fold_num = 3
seed = 5
np.random.seed(seed)

kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(x_test, y_test):
    # create model
    model = Sequential()

    # Dense：全結合のニューラルネットワークレイヤー
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(128, 128, 1)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu')) # 活性化関数：relu
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    #model.add(Dropout(0.2)) # 過学習防止用：入力の20%を0にする（破棄）
    model.add(layers.Dense(num_classes, activation='sigmoid')) # 活性化関数：softmax

    #model.summary()

    # コンパイル（多クラス分類問題）
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    # 構築したモデルで学習
    model.fit(x_test[train], np_utils.to_categorical(y_test[train], 2), batch_size=batch_size, epochs=epochs)

    y_pred = model.predict_classes(x_test[test])

    sum_all_j = 0
    sum_all_s = 0
    sum_j = 0
    sum_s = 0

    for i in range(x_test[test].shape[0]):
        #print("%s:%s" % (y_pred[i], Y_eva[test][i]))
        if y_pred[i] == 1:
            if y_pred[i] == y_test[test][i]:           
                sum_j = sum_j + 1  

            sum_all_j = sum_all_j + 1  

        else:
            if y_pred[i] == y_test[test][i]:
                sum_s = sum_s + 1

            sum_all_s = sum_all_s + 1


    print("人工物画像の正解枚数：",sum_j,"/",sum_all_j)
    print("自然物画像の正解枚数：",sum_s,"/",sum_all_s)
    
    sum_j =  sum_j / sum_all_j * 100
    sum_s =  sum_s / sum_all_s * 100
    ave = (sum_s + sum_j) / 2 + ave

    print("人工物画像の正解率＝",sum_j,"%")
    print("自然物画像の正解率＝",sum_s,"%")
    print("全体画像の正解率＝",(sum_s + sum_j) / 2,"%")
      
    # Evaluate
    scores = model.evaluate(x_test[test], keras.utils.to_categorical(y_test[test], num_classes), verbose=0)
    print(x_test[test].shape)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)
    cvscores.append((sum_s + sum_j) / 2)

print("平均正解率：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/100
1600/1600 [==============================] - 8s 5ms/step - loss: 2.2130 - acc: 0.5281
Epoch 2/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7473 - acc: 0.5375
Epoch 3/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7631 - acc: 0.5409
Epoch 4/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.6660 - acc: 0.6269
Epoch 5/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.5324 - acc: 0.7591
Epoch 6/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.4552 - acc: 0.8122
Epoch 7/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.4175 - acc: 0.8291
Epoch 8/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.3827 - acc: 0.8491
Epoch 9/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.3747 - acc: 0.8462
Epoch 10/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.3557 - acc: 0.8662
Epoch 11/100
1600/1

1600/1600 [==============================] - 5s 3ms/step - loss: 0.0214 - acc: 0.9916
Epoch 84/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0266 - acc: 0.9909
Epoch 85/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0448 - acc: 0.9878
Epoch 86/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0293 - acc: 0.9922
Epoch 87/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0183 - acc: 0.9950
Epoch 88/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0456 - acc: 0.9841
Epoch 89/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0296 - acc: 0.9891
Epoch 90/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0112 - acc: 0.9972
Epoch 91/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0414 - acc: 0.9925
Epoch 92/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0286 - acc: 0.9931
Epoch 93/100
1600/1600 

1600/1600 [==============================] - 5s 3ms/step - loss: 0.0818 - acc: 0.9778
Epoch 65/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0545 - acc: 0.9822
Epoch 66/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0343 - acc: 0.9875
Epoch 67/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0489 - acc: 0.9875
Epoch 68/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0239 - acc: 0.9938
Epoch 69/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0958 - acc: 0.9766
Epoch 70/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0398 - acc: 0.9872
Epoch 71/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0490 - acc: 0.9888
Epoch 72/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0437 - acc: 0.9853
Epoch 73/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0777 - acc: 0.9841
Epoch 74/100
1600/1600 

1600/1600 [==============================] - 5s 3ms/step - loss: 0.0525 - acc: 0.9816
Epoch 46/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0558 - acc: 0.9816
Epoch 47/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0722 - acc: 0.9838
Epoch 48/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0834 - acc: 0.9822
Epoch 49/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0607 - acc: 0.9813
Epoch 50/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0939 - acc: 0.9728
Epoch 51/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0587 - acc: 0.9822
Epoch 52/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0582 - acc: 0.9775
Epoch 53/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0428 - acc: 0.9853
Epoch 54/100
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0534 - acc: 0.9828
Epoch 55/100
1600/1600 